<a href="https://colab.research.google.com/github/mightyoctopus/amazon-pricer-model-open-source-fine-tuned-models/blob/main/pricer_model_merge_and_unload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merge LoRA Adaptor into Llama 3.1 8b

Base Model: meta-llama/Meta-Llama-3.1-8B

Adaptor: https://huggingface.co/MightyOctopus/product-pricer-2025-11-29_11.36.40


Merged Weights = W0 + BA

In [ ]:
!pip install -q --upgrade torch peft transformers huggingface_hub bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

from google.colab import userdata
from huggingface_hub import login

In [3]:
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
ADAPTER = "MightyOctopus/product-pricer-2025-11-29_11.36.40"

In [4]:
hf_token = userdata.get("HF_TOKEN")
login(hf_token, add_to_git_credential=True)

In [ ]:
### Load the base model

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    dtype=torch.bfloat16,
    device_map="cuda" ### Use cuda instead of "auto, to load all model shards in one place (GPU)
)

In [ ]:
### Load LoRA Adapter (Best performing version among all fine tuned LoRA adaptors)
### https://huggingface.co/MightyOctopus/product-pricer-2025-11-29_11.36.40

fine_tuned_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER
)

In [7]:
### Merge into base

merged_model = fine_tuned_model.merge_and_unload()

In [8]:
### Save the merged model and push to the hub

merged_model.save_pretrained("pricer_merged_model_A_v1")

In [ ]:
merged_model.push_to_hub("MightyOctopus/pricer-merged-model-A-v1")